# ASSIGNMENT 4 - RUNNING CONVOLUTION

In [ ]:
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from scipy import signal
import IPython as ip
import time

### Load the dry signal and the impulse response

##### make sure they are mono and have the same sample rate

In [ ]:
ir,sr_ir = sf.read('REW IR Studio A Control 16_44.wav')
crir,sr_crir = sf.read('Creative IR_01 16_44.wav')
gtr,sr_gtr = sf.read('Guitar.wav')

print(sr_ir,sr_gtr,sr_crir)

### Running convolution

##### You can use functions (and code) done in class to perform direct/fast convolutions within the Running Convolution algorithm. However, note that the assignment consists on the running convolution part.

In [ ]:
def dir_conv(x,h):
    
    c = np.zeros(len(x)+len(h)-1)
   
    for i in range (len(h)):
        c[i:(i+len(x))] += (x * h[i])
    
    return c

In [ ]:
def fast_conv(x,h):

    num_z_x = np.zeros(len(h)-1)
    x_z = np.hstack([x,num_z_x])

    num_z_h = np.zeros(len(x)-1)
    h_z = np.hstack([h,num_z_h])

    X = np.fft.fft(x_z)
    H = np.fft.fft(h_z)

    mult = X * H

    result = np.real(np.fft.ifft(mult))

    return result

In [ ]:
# [For Points 3,4 and 5 in pdf] Main algorithm

def running_convolver (ir, a_input, conv_m, b_size):
    
    # INPUTS
    # ir:  array containing the impulse response
    # a_input: array containing a signal
    # conv_m: convolution method:
    #         -direct convolution
    #         -fast convolution
    # b_size: length of input segments to be convolved with impulse response 

    # OUTPUTS:
    # ret_conv:         Post-convolution output signal vector
    
    #Type your code here:
    num_seg = int(np.ceil(len(a_input)/b_size))
    
    place_holder = np.zeros(len(a_input) + len(ir) - 1)

    for i in range(num_seg):
        st = b_size*i
        en = st+b_size
        if en > len(a_input):
            en = len(a_input)         

        buffer = a_input[st:en]
        
        if conv_m == 'direct': #do it directly
            place_holder[st:st + b_size + (len(ir)-1)] += dir_conv(buffer,ir)
        elif conv_m == 'fast': #do it with DFT, multiply, IDFT
            place_holder[st:st + b_size + (len(ir)-1)] += fast_conv(buffer,ir)
        else: print('Please choose "direct" or "fast"')

#         if conv_m == 'direct': #do it directly
#             conv_buff = dir_conv(buffer,ir)
#         elif conv_m == 'fast': #do it with DFT, multiply, IDFT
#             conv_buff = fast_conv(buffer,ir)
#         else: print('Please choose "direct" or "fast"')
        
#         en_c = st + b_size + (len(ir)-1)
        
#         place_holder[st:en_c] += conv_buff
        
    ret_conv = place_holder

    return ret_conv

### [For Points 3,4 and 5 in pdf] Use your function to convolve Guitar.wav against your IR using a segment length of 512

In [ ]:
b_size = 512
a_input = gtr
sr = 44100

conv_sig = running_convolver(ir, a_input, 'fast', b_size)

ip.display.display(ip.display.Audio(conv_sig, rate = sr))

### Normalize the result to avoid clipping

In [ ]:
conv_norm = conv_sig/np.max(np.abs(conv_sig))

### Export it

In [ ]:
sf.write('ConvolvedSignal.wav',conv_norm,samplerate = sr)

## [Point 6 in pdf] Copy your running_convolver function from above and modify it to:
- Playback buffers of exactly 8192 samples of the guitar audio convolved against the IR. 
- Display the operation time for each buffer should also be displayed

In [ ]:
def playback_convolver(ir, a_input, conv_m, b_size):
    
    # INPUTS
    # ir:  array containing the impulse response
    # a_input: array containing a signal
    # conv_m: convolution method:
    #         -direct convolution
    #         -fast convolution
    # b_size: length of input segments to be convolved with impulse response 

    # OUTPUTS:
    # ret_conv:         Post-convolution output signal vector
    
    #Type your code here:
    num_seg = int(np.ceil(len(a_input)/b_size))
    
    place_holder = np.zeros(len(a_input) + len(ir) - 1)

    for i in range(num_seg):
        st = b_size*i
        en = st+b_size
        if en > len(a_input):
            en = len(a_input)

        buffer = a_input[st:en]
      
        if conv_m == 'direct': #do it directly
            conv_buff = dir_conv(buffer,ir)
        elif conv_m == 'fast': #do it with DFT, multiply, IDFT
            conv_buff = fast_conv(buffer,ir)
        else: print('Please choose "direct" or "fast"')

        en_c = st + b_size + (len(ir)-1)
        
        place_holder[st:en_c] += conv_buff
    
    pb_len = int(np.ceil(len(place_holder)/8192))
    
    for j in range(pb_len):
        start = time.time()
            
        st = 8192*j
        en = st+8192
        if en > len(place_holder):
            pad = np.zeros(8192-(len(place_holder)-st))
            place_holder = np.hstack([place_holder,pad])
            en = st+8192
            
        pb_buff = np.zeros(8192)
        
        pb_buff += place_holder[st:en]
        
        ip.display.display(ip.display.Audio(pb_buff, rate = 44100))
    
        stop = time.time()
        print(stop-start)
        print('---------')
        
    return 'Behold the answer to part 6a below!'

In [ ]:
playback_convolver(ir, a_input, 'fast', b_size)

##### This would be relevant in a real-time audio application because you can see if your code is efficient enough to run through these buffers fast enough to party in real-time.

## Extra Credit

In [ ]:
b_size = 512
a_input = gtr
sr = 44100

conv_sig = running_convolver(crir, a_input, 'fast', b_size)

ip.display.display(ip.display.Audio(conv_sig, rate = sr))